In [ ]:
pip install openpyxl

In [ ]:
pip install librosa openpyxl pandas numpy

In [ ]:
pip install --upgrade youtube-dl

In [ ]:
pip install librosa --upgrade

In [ ]:
pip install yt-dlp

In [ ]:
import subprocess
import os

def download_audio_from_youtube(video_url, output_filename):
    try:
        # Step 1: Download the audio using yt-dlp
        audio_file = 'temp_audio.m4a'
        command = ['yt-dlp', '-f', 'bestaudio', '--extract-audio', '--audio-format', 'm4a', '-o', audio_file, video_url]
        subprocess.run(command, check=True)
        
        # Step 2: Convert the downloaded audio to WAV using ffmpeg
        command = ['ffmpeg', '-i', audio_file, output_filename]
        subprocess.run(command, check=True)
        
        # Clean up temporary audio file
        os.remove(audio_file)
        
        print(f'Audio has been downloaded and saved as {output_filename}')
    except subprocess.CalledProcessError as e:
        print(f'An error occurred during the subprocess execution: {e}')
    except Exception as e:
        print(f'An error occurred: {e}')

# Replace 'video_url' with a valid YouTube video URL
# video_url = 'https://youtu.be/DTcJmIbn5nw?si=RJN6X7BWr9jr7sp0'    #place the video url here
# output_filename = 'AUDIO.wav'                    #location where audio is saved

download_audio_from_youtube(video_url, output_filename)

In [ ]:
import librosa
import numpy as np

# Load the audio file
# y, sr = librosa.load('AUDIO.wav')                #location where audio is saved

# Extract audio features
# Pitch
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
pitch = np.mean(pitches[pitches > 0])

# Tempo
tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

# Zero-Crossing Rate
zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y)[0])

# Spectral Centroid
spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)[0])

# Chroma Mean
chroma_mean = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)

In [ ]:
print(f'Pitch: {pitch}')
print(f'Tempo (BPM): {tempo}')
print(f'Zero-Crossing Rate: {zero_crossing_rate}')
print(f'Spectral Centroid: {spectral_centroid}')
print(f'Chroma Mean: {chroma_mean}')

In [119]:
# Define emotions and their energy-based weights
['Anger','Surprised','Happy','Neutral','Disgust', 'Fear', 'Sad' ]
weights = [7,6,5,4,3,2,1] # Hypothetical weights based on energy

In [ ]:
# Print each emotion along with its weight
for emotion, weight in zip(emotions, weights):
    print(f'Emotion: {emotion}, Weight: {weight}')

# Calculate the total weight of all emotions
total_weight = sum(weights)
print(f'Total Weight: {total_weight}')

# Find the emotion with the highest weight
max_weight_index = weights.index(max(weights))
max_emotion = emotions[max_weight_index]
print(f'Emotion with Maximum Weight: {max_emotion}')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define emotions and their weights
emotions = ['Anger','Surprised','Happy','Neutral','Disgust', 'Fear', 'Sad' ]
weights = [7,6,5,4,3,2,1]

# Define audio features and their values
audio_features = {
    'Pitch': [1885.0869140625],
    'Tempo (BPM)': [112.34714674],
    'Zero-Crossing Rate': [0.19199533794045454],
    'Spectral Centroid': [2907.1634658754288],
    'Chroma Mean': [np.mean([0.40174642, 0.35858825, 0.35553306, 0.35822368, 0.32790217,
                             0.33791435, 0.3739753,  0.38466904, 0.39902058, 0.41803277,
                             0.4354073, 0.44042513])]
}

# Create a DataFrame for audio features
df_audio = pd.DataFrame(audio_features)

# Compute the correlation matrix based on weights and audio feature values
correlation_matrix = np.zeros((len(emotions), len(df_audio.columns)))

# Populate the correlation matrix using provided weights and feature values
for i, emotion in enumerate(emotions):
    for j, feature in enumerate(df_audio.columns):
        if feature == 'Chroma Mean':
            # Use the mean value for Chroma Mean feature
            feature_value = df_audio[feature].iloc[0]
        else:
            # Use the single value for other features
            feature_value = df_audio[feature].iloc[0]
        # Compute correlation based on provided weights and feature values
        correlation_matrix[i, j] = weights[i] * feature_value

# Calculate the maximum absolute value in correlation_matrix for normalization
max_abs_value = np.max(np.abs(correlation_matrix))

# Normalize correlation_matrix between -1 and 1
correlation_matrix_normalized = correlation_matrix / max_abs_value

# Create a DataFrame for the normalized correlation matrix
df_corr = pd.DataFrame(correlation_matrix_normalized, index=emotions, columns=df_audio.columns)

# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plot the correlation matrix
heatmap = sns.heatmap(df_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, linewidths=.5, square=True)

# Adjust ticks and labels to ensure equal scale
plt.xticks(np.arange(len(df_audio.columns)) + 0.5, df_audio.columns, rotation=45)
plt.yticks(np.arange(len(emotions)) + 0.5, emotions, rotation=0)
plt.tick_params(axis='both', which='both', length=0)  # remove ticks

# Add color bar using the mappable object returned by sns.heatmap
cbar = heatmap.collections[0].colorbar
cbar.set_label('Correlation')  # Add label to the color bar

# Add title and show plot
plt.title('Correlation Matrix Between Emotions and Audio Features')
plt.tight_layout()
plt.show()